<a href="https://colab.research.google.com/github/mcasivaram/Datascience/blob/main/null_innovation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data=pd.read_csv("/content/drive/MyDrive/Tweet_NFT.xlsx - Sheet1.csv")

In [ ]:
df1=pd.DataFrame(data)

In [ ]:
df1['tweet_intent'].count()

In [ ]:
df1['tweet_intent'].isnull().sum()

In [ ]:
df1=df1.dropna()  

In [ ]:
df1=df1.drop(['id','tweet_created_at'],axis=1)

In [ ]:
df1['tweet_text']

In [ ]:
df1['tweet_processed'] = df1['tweet_text'].str.replace("[^a-zA-Z]", " ")

In [ ]:
df1['tweet_processed'] = df1['tweet_processed'].apply(lambda row: ' '.join([word for word in row.split() if len(word)>2]))

In [ ]:
df1['tweet_processed']

In [ ]:
df1['tweet_processed'] = [tweet.lower() for tweet in df1['tweet_processed']]

In [ ]:
df1['tweet_processed']

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk import word_tokenize
stop_words = stopwords.words('english')
# Function to remove stop words 
def remove_stopwords(tweet):
    # iNPUT : IT WILL TAKE ROW/REVIEW AS AN INPUT
    # take the paragraph, break into words, check if the word is a stop word, remove if stop word, combine the words into a para again
    tweet_tokenized = word_tokenize(tweet)
    tweet_new = " ".join([i for i in tweet_tokenized  if i not in stop_words])
    return tweet_new
df1['tweet_processed'] = [remove_stopwords(tweet) for tweet in df1['tweet_processed']]


In [ ]:
df1['tweet_processed']

In [ ]:
df1['tweet_processed']

In [ ]:
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

# function to convert nltk tag to wordnet tag
lemmatizer = WordNetLemmatizer()

# Finds the part of speech tag
# Convert the detailed POS tag into a shallow information
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

# lemmatize sentence using pos tag
def lemmatize_sentence(sentence):
  # word tokenize -> pos tag (detailed) -> wordnet tag (shallow pos) -> lemmatizer -> root word
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)


df1['tweet_processed'] = df1['tweet_processed'].apply(lambda x: lemmatize_sentence(x))

In [ ]:
df1['tweet_intent'].unique()

In [ ]:
# Importing module
from sklearn.feature_extraction.text import TfidfVectorizer
import sklearn
# Creating matrix of top 2500 tokens
tfidf = TfidfVectorizer(max_features=2500)
# tmp_df = tfidf.fit_transform(df.review_processed)
# feature_names = tfidf.get_feature_names()
# pd.DataFrame(tmp_df.toarray(), columns = feature_names).head() 

X = tfidf.fit_transform(df1.tweet_processed).toarray()
y = pd.get_dummies(df1,columns=['tweet_intent'])
featureNames = tfidf.get_feature_names()


X, y


# Splitting the dataset into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)


In [ ]:
X

In [ ]:
y

In [ ]:
X_train.shape, X_test.shape

In [ ]:
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score

In [ ]:
for lr in [0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1,0.11,0.12,0.13,0.14,0.15,0.2,0.5,0.7,1]:
  model = xgb.XGBRegressor(learning_rate = lr, n_estimators=100, verbosity = 0) # initialise the model
  model.fit(X_train,y_train) #train the model
  model.score(X_test, y_test) # scoring the model - r2 squared
  print("Learning rate : ", lr, " Train score : ", model.score(X_train,y_train), " Cross-Val score : ", np.mean(cross_val_score(model, X_train, y_train, cv=10)))
